In [70]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
import datetime
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.austintexas.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.austintexas.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("fdj4-gpfu", limit=200000,)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [71]:
results_df.head()

,incident_report_number,crime_type,ucr_code,family_violence,occ_date_time,occ_date,occ_time,rep_date_time,rep_date,rep_time,...,longitude,location,:@computed_region_a3it_2a2z,:@computed_region_8spj_utxs,:@computed_region_q9nd_rr82,:@computed_region_qwte_z96m,ucr_category,category_description,clearance_status,clearance_date
0,2022813090,ASSAULT BY CONTACT,902,N,2022-01-08T23:57:00.000,2022-01-08T00:00:00.000,2357,2022-01-08T23:57:00.000,2022-01-08T00:00:00.000,2357,...,-97.73818499,"{'latitude': '30.25953309', 'longitude': '-97....",2856,9,10,1405,NaN,NaN,NaN,NaN
1,202281353,FAMILY DISTURBANCE,3400,N,2022-01-08T23:51:00.000,2022-01-08T00:00:00.000,2351,2022-01-09T00:04:00.000,2022-01-09T00:00:00.000,4,...,-97.71813346,"{'latitude': '30.21951765', 'longitude': '-97....",3256,3,3,1599,NaN,NaN,NaN,NaN
2,202281199,AGG ROBBERY/DEADLY WEAPON,300,N,2022-01-08T23:37:00.000,2022-01-08T00:00:00.000,2337,2022-01-08T23:37:00.000,2022-01-08T00:00:00.000,2337,...,-97.71524279,"{'latitude': '30.42489395', 'longitude': '-97....",3243,7,7,3749,120,Robbery,NaN,NaN
3,202281282,FAMILY DISTURBANCE,3400,N,2022-01-08T23:34:00.000,2022-01-08T00:00:00.000,2334,2022-01-08T23:34:00.000,2022-01-08T00:00:00.000,2334,...,-97.72062928,"{'latitude': '30.22899126', 'longitude': '-97....",3256,3,3,102,NaN,NaN,NaN,NaN
4,202281324,AUTO THEFT,700,N,2022-01-08T23:31:00.000,2022-01-08T00:00:00.000,2331,2022-01-08T23:31:00.000,2022-01-08T00:00:00.000,2331,...,-97.71745933,"{'latitude': '30.35094052', 'longitude': '-97....",3645,7,7,NaN,240,Auto Theft,NaN,NaN


In [72]:
results_df.dtypes

incident_report_number         object
crime_type                     object
ucr_code                       object
family_violence                object
occ_date_time                  object
occ_date                       object
occ_time                       object
rep_date_time                  object
rep_date                       object
rep_time                       object
location_type                  object
address                        object
zip_code                       object
council_district               object
sector                         object
district                       object
pra                            object
census_tract                   object
x_coordinate                   object
y_coordinate                   object
latitude                       object
longitude                      object
location                       object
:@computed_region_a3it_2a2z    object
:@computed_region_8spj_utxs    object
:@computed_region_q9nd_rr82    object
:@computed_r

In [75]:
crime_df = results_df[['incident_report_number','crime_type', 'family_violence', 'occ_date', 'occ_time', 'zip_code', 'latitude', 'longitude']]
crime_df.dropna(inplace=True)
crime_df.head()

C:\Users\Christopher Garcia\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,incident_report_number,crime_type,family_violence,occ_date,occ_time,zip_code,latitude,longitude
0,2022813090,ASSAULT BY CONTACT,N,2022-01-08T00:00:00.000,2357,78701,30.25953309,-97.73818499
1,202281353,FAMILY DISTURBANCE,N,2022-01-08T00:00:00.000,2351,78741,30.21951765,-97.71813346
2,202281199,AGG ROBBERY/DEADLY WEAPON,N,2022-01-08T00:00:00.000,2337,78727,30.42489395,-97.71524279
3,202281282,FAMILY DISTURBANCE,N,2022-01-08T00:00:00.000,2334,78741,30.22899126,-97.72062928
4,202281324,AUTO THEFT,N,2022-01-08T00:00:00.000,2331,78757,30.35094052,-97.71745933


In [76]:
#Converting time data to datetime

timefmt_date = "%Y-%m-%d"
timefmt_time = "%H:%M" 
crime_df['occ_date']= pd.to_datetime(crime_df['occ_date'], format = timefmt_date, errors='coerce')

time_occ = []

# Loop to format data for date time
for str in crime_df['occ_time']:
          if len(str) == 4 :
                    val =  str[:2] + ':' +str[2:]
          elif len(str) == 3:
                    val = str[:1] + ':' +str[1:]
          elif len(str) == 2:
                    val = '00:' + str
          elif len(str) == 1:
                    val = '00:0' + str
          time_occ.append(val)
#
crime_df['time_occ'] = time_occ

crime_df['time_occ'] = pd.to_datetime(crime_df['time_occ'], format = timefmt_time, errors='coerce')
# crime_df['time_occ'] = [time.time() for time in crime_df['time_occ']]
crime_df.head()
          

C:\Users\Christopher Garcia\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Christopher Garcia\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Christopher Garcia\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

,incident_report_number,crime_type,family_violence,occ_date,occ_time,zip_code,latitude,longitude,time_occ
0,2022813090,ASSAULT BY CONTACT,N,2022-01-08,2357,78701,30.25953309,-97.73818499,1900-01-01 23:57:00
1,202281353,FAMILY DISTURBANCE,N,2022-01-08,2351,78741,30.21951765,-97.71813346,1900-01-01 23:51:00
2,202281199,AGG ROBBERY/DEADLY WEAPON,N,2022-01-08,2337,78727,30.42489395,-97.71524279,1900-01-01 23:37:00
3,202281282,FAMILY DISTURBANCE,N,2022-01-08,2334,78741,30.22899126,-97.72062928,1900-01-01 23:34:00
4,202281324,AUTO THEFT,N,2022-01-08,2331,78757,30.35094052,-97.71745933,1900-01-01 23:31:00


In [79]:
crime_df.groupby(['crime_type']).count().sample(5)

,incident_report_number,family_violence,occ_date,occ_time,zip_code,latitude,longitude,time_occ
crime_type,,,,,,,,
VOYEURISM-PEEPING TOM,10,10,10,10,10,10,10,10
PURCHASING PROSTITUTION,7,7,7,7,7,7,7,7
DISRUPTIVE ACTS AT SCHOOLS,1,1,1,1,1,1,1,1
SUSPICIOUS PERSON,1973,1973,1973,1973,1973,1973,1973,1973
FORGERY OF IDENTIFICATION,19,19,19,19,19,19,19,19


In [80]:
crime_df = crime_df.loc[(crime_df['occ_date']>= '2020-01-01') & (crime_df['occ_date']< '2022-01-01')]
crime_df

,incident_report_number,crime_type,family_violence,occ_date,occ_time,zip_code,latitude,longitude,time_occ
1115,20213651544,PUBLIC INTOXICATION,N,2021-12-31,2350,78701,30.2672999,-97.73885705,1900-01-01 23:50:00
1116,20213651545,ASSAULT ON PUBLIC SERVANT,N,2021-12-31,2350,78701,30.26373894,-97.74365146,1900-01-01 23:50:00
1117,20225000622,THEFT,N,2021-12-31,2350,78741,30.21526412,-97.70301942,1900-01-01 23:50:00
1118,20213651542,DOC DISCHARGE GUN - PUB PLACE,N,2021-12-31,2347,78752,30.32804875,-97.69368286,1900-01-01 23:47:00
1119,202210091,AGG ASLT STRANGLE/SUFFOCATE,Y,2021-12-31,2340,78758,30.36926216,-97.69510501,1900-01-01 23:40:00
...,...,...,...,...,...,...,...,...,...
195555,20208000084,THEFT OF BICYCLE,N,2020-01-01,0,78703,30.27315833,-97.76014527,1900-01-01 00:00:00
195556,20205005261,HARASSMENT,N,2020-01-01,0,78741,30.2156655,-97.70354685,1900-01-01 00:00:00
195557,20206900041,THEFT FROM AUTO,N,2020-01-01,0,78741,30.22428901,-97.68704573,1900-01-01 00:00:00
195558,20215010247,IDENTITY THEFT,N,2020-01-01,0,78736,30.23947683,-97.89463328,1900-01-01 00:00:00


In [81]:
#drop old time column. 
crime_df.drop('occ_time',axis = 1,inplace=True)

In [82]:
crime_df

,incident_report_number,crime_type,family_violence,occ_date,zip_code,latitude,longitude,time_occ
1115,20213651544,PUBLIC INTOXICATION,N,2021-12-31,78701,30.2672999,-97.73885705,1900-01-01 23:50:00
1116,20213651545,ASSAULT ON PUBLIC SERVANT,N,2021-12-31,78701,30.26373894,-97.74365146,1900-01-01 23:50:00
1117,20225000622,THEFT,N,2021-12-31,78741,30.21526412,-97.70301942,1900-01-01 23:50:00
1118,20213651542,DOC DISCHARGE GUN - PUB PLACE,N,2021-12-31,78752,30.32804875,-97.69368286,1900-01-01 23:47:00
1119,202210091,AGG ASLT STRANGLE/SUFFOCATE,Y,2021-12-31,78758,30.36926216,-97.69510501,1900-01-01 23:40:00
...,...,...,...,...,...,...,...,...
195555,20208000084,THEFT OF BICYCLE,N,2020-01-01,78703,30.27315833,-97.76014527,1900-01-01 00:00:00
195556,20205005261,HARASSMENT,N,2020-01-01,78741,30.2156655,-97.70354685,1900-01-01 00:00:00
195557,20206900041,THEFT FROM AUTO,N,2020-01-01,78741,30.22428901,-97.68704573,1900-01-01 00:00:00
195558,20215010247,IDENTITY THEFT,N,2020-01-01,78736,30.23947683,-97.89463328,1900-01-01 00:00:00


In [83]:
# Convert latitude and longitude as float
crime_df['latitude'] = crime_df['latitude'].astype('float')
crime_df['longitude'] = crime_df['longitude'].astype('float')
crime_df.dtypes



incident_report_number            object
crime_type                        object
family_violence                   object
occ_date                  datetime64[ns]
zip_code                          object
latitude                         float64
longitude                        float64
time_occ                  datetime64[ns]
dtype: object

In [84]:
# Converting zip_codes to integers
crime_df['zip_code'] = crime_df['zip_code'].astype('int64')
crime_df.dtypes


incident_report_number            object
crime_type                        object
family_violence                   object
occ_date                  datetime64[ns]
zip_code                           int64
latitude                         float64
longitude                        float64
time_occ                  datetime64[ns]
dtype: object

In [85]:
# Reorganize column info

column_order = ['incident_report_number','crime_type', 'family_violence', 'occ_date', 'time_occ', 'zip_code','latitude','longitude']
crime_df = crime_df[column_order]
crime_df.groupby(['zip_code']).count().head()

,incident_report_number,crime_type,family_violence,occ_date,time_occ,latitude,longitude
zip_code,,,,,,,
78610,10,10,10,10,10,10,10
78613,866,866,866,866,866,866,866
78616,1,1,1,1,1,1,1
78617,1661,1661,1661,1661,1661,1661,1661
78641,2,2,2,2,2,2,2


In [86]:
#grouping crime type by zipcode

crime_by_zipcode_df = crime_df.groupby(['zip_code', 'crime_type']).count()
crime_by_zipcode_df

incident_report_number  \
zip_code crime_type                                               
78610    ASSAULT W/INJURY-FAM/DATE VIOL                       1   
         CRED CARD ABUSE - OTHER                              1   
         DWI                                                  1   
         DWI 2ND                                              1   
         EVADING / VEHICLE PURSUIT                            1   
...                                                         ...   
78759    VIOL STATE LAW - OTHER                               4   
         VIOL TEMP EX PARTE  ORDER                            1   
         VOYEURISM-PEEPING TOM                                1   
         WARRANT ARREST NON TRAFFIC                          10   
         WEAPON VIOL - OTHER                                  1   

                                         family_violence  occ_date  time_occ  \
zip_code crime_type                                                            
78610    ASSAULT W/INJURY-FAM/DATE VIOL                1         1         1   
         CRED CARD ABUSE - OTHER                       1         1         1   
         DWI                                           1         1         1   
         DWI 2ND                                       1         1         1   
         EVADING / VEHICLE PURSUIT                     1         1         1   
...                                                  ...       ...       ...   
78759    VIOL STATE LAW - OTHER                        4         4         4   
         VIOL TEMP EX PARTE  ORDER                     1         1         1   
         VOYEURISM-PEEPING TOM                         1         1         1   
         WARRANT ARREST NON TRAFFIC                   10        10        10   
         WEAPON VIOL - OTHER                           1         1         1   

                                         latitude  longitude  
zip_code crime_type                                           
78610    ASSAULT W/INJURY-FAM/DATE VIOL         1          1  
         CRED CARD ABUSE - OTHER                1          1  
         DWI                                    1          1  
         DWI 2ND                                1          1  
         EVADING / VEHICLE PURSUIT              1          1  
...                                           ...        ...  
78759    VIOL STATE LAW - OTHER                 4          4  
         VIOL TEMP EX PARTE  ORDER              1          1  
         VOYEURISM-PEEPING TOM                  1          1  
         WARRANT ARREST NON TRAFFIC            10         10  
         WEAPON VIOL - OTHER                    1          1  

[5872 rows x 6 columns]

In [88]:
import pandas as pd
zip_code_avg_inc = pd.read_csv('Avg_INcome_by_Zip_Code_zip_atlas.csv')
zip_code_avg_inc.head()

,#,Zip Code,Location,City,Population,Avg. Income/H/hold,National Rank
0,1,78730,"30.363632, -97.850355","Austin, Texas","4,885","$128,524.00",#58
1,2,78732,"30.382536, -97.894904","Austin, Texas","3,629","$103,951.00",#209
2,3,78739,"30.178839, -97.889064","Austin, Texas","8,643","$102,707.00",#227
3,4,78738,"30.327510, -97.989040","Austin, Texas","2,840","$102,295.00",#231
4,5,78733,"30.323857, -97.894860","Austin, Texas","8,716","$102,239.00",#232


In [89]:
result = pd.merge(crime_df, zip_code_avg_inc, how="inner", left_on=['zip_code'], right_on=['Zip Code'])
result.drop(['Zip Code', 'City', 'Location', '#'], axis = 1, inplace=True)
result = result[['incident_report_number','crime_type', 'family_violence', 'occ_date', 'time_occ', 'zip_code', 'Population', 'Avg. Income/H/hold', 'National Rank', 'latitude' , 'longitude']]
result


,incident_report_number,crime_type,family_violence,occ_date,time_occ,zip_code,Population,Avg. Income/H/hold,National Rank,latitude,longitude
0,20213651544,PUBLIC INTOXICATION,N,2021-12-31,1900-01-01 23:50:00,78701,"3,855","$35,757.00","#16,509",30.267300,-97.738857
1,20213651545,ASSAULT ON PUBLIC SERVANT,N,2021-12-31,1900-01-01 23:50:00,78701,"3,855","$35,757.00","#16,509",30.263739,-97.743651
2,20213651508,PUBLIC INTOXICATION,N,2021-12-31,1900-01-01 23:21:00,78701,"3,855","$35,757.00","#16,509",30.269900,-97.748559
3,20225000001,BURGLARY OF VEHICLE,N,2021-12-31,1900-01-01 21:45:00,78701,"3,855","$35,757.00","#16,509",30.268218,-97.738693
4,20213650837,RESISTING ARREST OR SEARCH,N,2021-12-31,1900-01-01 15:20:00,78701,"3,855","$35,757.00","#16,509",30.264693,-97.737904
...,...,...,...,...,...,...,...,...,...,...,...
186034,20201211001,ASSAULT ON PUBLIC SERVANT,N,2020-04-30,1900-01-01 16:54:00,78733,"8,716","$102,239.00",#232,30.339078,-97.884978
186035,20201011302,BOATING WHILE INTOXICATED,N,2020-04-10,1900-01-01 20:50:00,78733,"8,716","$102,239.00",#232,30.353315,-97.858720
186036,20205014278,BURGLARY OF VEHICLE,N,2020-04-10,1900-01-01 13:40:00,78733,"8,716","$102,239.00",#232,30.339078,-97.884978
186037,20205013410,CHILD CUSTODY INTERFERE,N,2020-04-03,1900-01-01 18:00:00,78733,"8,716","$102,239.00",#232,30.334920,-97.870245


In [90]:
# Read food_final_tx.csv
census_zip = pd.read_csv('food_final_tx.csv')
census_zip = census_zip.sort_values(['ZIP'])
census_zip.head()

,CensusTract,State,County,Urban,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,ZIP
10709,48453002307,Texas,Travis County,1,1,1,1,1,73301
10734,48453002411,Texas,Travis County,1,1,1,1,0,73301
9316,48421950200,Texas,Sherman County,0,0,0,0,0,73960
2756,48113013616,Texas,Dallas County,1,0,0,0,0,75001
2796,48113013725,Texas,Dallas County,1,0,1,0,1,75001


In [91]:
#group census zones by zip codes and sum the food deficient zones
census_grouping = census_zip.groupby(['ZIP'],as_index=False).sum()
census_grouping.head()


,ZIP,CensusTract,Urban,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle
0,73301,96906004718,2,2,2,2,1
1,73960,48421950200,0,0,0,0,0
2,75001,288678082405,6,0,1,0,1
3,75002,673190439628,14,1,1,1,1
4,75006,673582199329,14,0,6,0,1


In [92]:
# Add total count of food deprived tracts. 
census_grouping['Low_access_tracts'] = census_grouping['LILATracts_1And10'] + census_grouping['LILATracts_halfAnd10'] + census_grouping['LILATracts_1And20'] + census_grouping['LILATracts_Vehicle']
census_grouping.head()

,ZIP,CensusTract,Urban,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,Low_access_tracts
0,73301,96906004718,2,2,2,2,1,7
1,73960,48421950200,0,0,0,0,0,0
2,75001,288678082405,6,0,1,0,1,2
3,75002,673190439628,14,1,1,1,1,4
4,75006,673582199329,14,0,6,0,1,7


In [93]:
# Calculate the total tracts WITHOUT changing the order

total_tracts = census_zip['ZIP'].value_counts().sort_index()*4
total_tracts

73301     8
73960     4
75001    24
75002    56
75006    56
         ..
79955     4
79968     4
79995     4
79996     4
79997     4
Name: ZIP, Length: 2425, dtype: int64

In [94]:
# get total count of tracts per zip code
total = pd.DataFrame(total_tracts, census_grouping['ZIP']).set_index(census_grouping.index)
total



,ZIP
0,8
1,4
2,24
3,56
4,56
...,...
2420,4
2421,4
2422,4
2423,4


In [96]:
# total_tracts added to the dataframe.

census_grouping['total_tracts'] = (total)
census_grouping.head(10)


,ZIP,CensusTract,Urban,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,Low_access_tracts,total_tracts
0,73301,96906004718,2,2,2,2,1,7,8
1,73960,48421950200,0,0,0,0,0,0,4
2,75001,288678082405,6,0,1,0,1,2,24
3,75002,673190439628,14,1,1,1,1,4,56
4,75006,673582199329,14,0,6,0,1,7,56
5,75007,1058602462151,22,2,5,2,1,10,88
6,75009,240461141218,2,0,0,0,0,0,20
7,75010,481210216271,10,0,0,0,0,0,40
8,75011,96234035351,2,0,0,0,0,0,8
9,75013,288510189061,6,1,1,1,1,4,24


In [97]:
census_grouping['Percentage_dep'] = (census_grouping['Low_access_tracts']/census_grouping['total_tracts']*100)


# Initiate list 
food_deprived_50 =[]
food_deprived_25 =[]
food_deprived_10 = []

# For loop to catogorize zip code food disparity.
for i in census_grouping['Percentage_dep']:
          if i >= 50:
                    i = 'Yes'
          else:
                    i='No'
          food_deprived_50.append(i)

for i in census_grouping['Percentage_dep']:
          if i >= 25:
                    i = 'Yes'
          else:
                    i='No'
          food_deprived_25.append(i)

for i in census_grouping['Percentage_dep']:
          if i >= 10:
                    i = 'Yes'
          else:
                    i='No'
          food_deprived_10.append(i)


census_grouping['food_deprived_50'] = food_deprived_50
census_grouping['food_deprived_25'] = food_deprived_25
census_grouping['food_deprived_10'] = food_deprived_10

census_grouping
                    

,ZIP,CensusTract,Urban,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,Low_access_tracts,total_tracts,Percentage_dep,food_deprived_50,food_deprived_25,food_deprived_10
0,73301,96906004718,2,2,2,2,1,7,8,87.500000,Yes,Yes,Yes
1,73960,48421950200,0,0,0,0,0,0,4,0.000000,No,No,No
2,75001,288678082405,6,0,1,0,1,2,24,8.333333,No,No,No
3,75002,673190439628,14,1,1,1,1,4,56,7.142857,No,No,No
4,75006,673582199329,14,0,6,0,1,7,56,12.500000,No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420,79955,48141001700,1,0,1,0,0,1,4,25.000000,No,Yes,Yes
2421,79968,48141001502,1,0,0,0,0,0,4,0.000000,No,No,No
2422,79995,48141003000,1,0,1,0,1,2,4,50.000000,Yes,Yes,Yes
2423,79996,48141010326,1,0,0,0,0,0,4,0.000000,No,No,No


In [98]:
# Keep only the info we need
cleaned_census_data = census_grouping[['ZIP', 'Low_access_tracts', 'total_tracts', 'Percentage_dep','food_deprived_50','food_deprived_25','food_deprived_10']]
cleaned_census_data.head(20)

,ZIP,Low_access_tracts,total_tracts,Percentage_dep,food_deprived_50,food_deprived_25,food_deprived_10
0,73301,7,8,87.500000,Yes,Yes,Yes
1,73960,0,4,0.000000,No,No,No
2,75001,2,24,8.333333,No,No,No
3,75002,4,56,7.142857,No,No,No
4,75006,7,56,12.500000,No,No,Yes
5,75007,10,88,11.363636,No,No,Yes
6,75009,0,20,0.000000,No,No,No
7,75010,0,40,0.000000,No,No,No
8,75011,0,8,0.000000,No,No,No
9,75013,4,24,16.666667,No,No,Yes


In [103]:
#Merge data frames together 

crime_df_cleaner = pd.merge(result, cleaned_census_data, how="inner", left_on=['zip_code'], right_on=['ZIP'])
crime_df_cleaner.head()

,incident_report_number,crime_type,family_violence,occ_date,time_occ,zip_code,Population,Avg. Income/H/hold,National Rank,latitude,longitude,ZIP,Low_access_tracts,total_tracts,Percentage_dep,food_deprived_50,food_deprived_25,food_deprived_10
0,20213651544,PUBLIC INTOXICATION,N,2021-12-31,1900-01-01 23:50:00,78701,"3,855","$35,757.00","#16,509",30.267300,-97.738857,78701,7,32,21.875,No,No,Yes
1,20213651545,ASSAULT ON PUBLIC SERVANT,N,2021-12-31,1900-01-01 23:50:00,78701,"3,855","$35,757.00","#16,509",30.263739,-97.743651,78701,7,32,21.875,No,No,Yes
2,20213651508,PUBLIC INTOXICATION,N,2021-12-31,1900-01-01 23:21:00,78701,"3,855","$35,757.00","#16,509",30.269900,-97.748559,78701,7,32,21.875,No,No,Yes
3,20225000001,BURGLARY OF VEHICLE,N,2021-12-31,1900-01-01 21:45:00,78701,"3,855","$35,757.00","#16,509",30.268218,-97.738693,78701,7,32,21.875,No,No,Yes
4,20213650837,RESISTING ARREST OR SEARCH,N,2021-12-31,1900-01-01 15:20:00,78701,"3,855","$35,757.00","#16,509",30.264693,-97.737904,78701,7,32,21.875,No,No,Yes


In [104]:
#formatting new data frame
crime_df_cleaner.drop('ZIP',axis=1, inplace=True)
crime_df_cleaner.drop_duplicates(inplace=True)
crime_df_cleaner.head()

,incident_report_number,crime_type,family_violence,occ_date,time_occ,zip_code,Population,Avg. Income/H/hold,National Rank,latitude,longitude,Low_access_tracts,total_tracts,Percentage_dep,food_deprived_50,food_deprived_25,food_deprived_10
0,20213651544,PUBLIC INTOXICATION,N,2021-12-31,1900-01-01 23:50:00,78701,"3,855","$35,757.00","#16,509",30.267300,-97.738857,7,32,21.875,No,No,Yes
1,20213651545,ASSAULT ON PUBLIC SERVANT,N,2021-12-31,1900-01-01 23:50:00,78701,"3,855","$35,757.00","#16,509",30.263739,-97.743651,7,32,21.875,No,No,Yes
2,20213651508,PUBLIC INTOXICATION,N,2021-12-31,1900-01-01 23:21:00,78701,"3,855","$35,757.00","#16,509",30.269900,-97.748559,7,32,21.875,No,No,Yes
3,20225000001,BURGLARY OF VEHICLE,N,2021-12-31,1900-01-01 21:45:00,78701,"3,855","$35,757.00","#16,509",30.268218,-97.738693,7,32,21.875,No,No,Yes
4,20213650837,RESISTING ARREST OR SEARCH,N,2021-12-31,1900-01-01 15:20:00,78701,"3,855","$35,757.00","#16,509",30.264693,-97.737904,7,32,21.875,No,No,Yes


In [118]:
#Formatting different columns.

#Remove # from national rank
crime_df_cleaner['National Rank'] = crime_df_cleaner['National Rank'].replace({'\#':''},regex = True)
crime_df_cleaner['National Rank'] = crime_df_cleaner['National Rank'].replace({'\,':''},regex = True).astype('int')

# Round food deprived to the hundreths place
crime_df_cleaner['Percentage_dep']= crime_df_cleaner['Percentage_dep'].round(2)

# Remove , and $ from average household value
crime_df_cleaner['Avg. Income/H/hold'] = crime_df_cleaner['Avg. Income/H/hold'].replace({'\$':''}, regex = True)
crime_df_cleaner['Avg. Income/H/hold'] = crime_df_cleaner['Avg. Income/H/hold'].replace({'\,':''}, regex = True).astype('float')

#Remove , from population 
crime_df_cleaner['Population'] = crime_df_cleaner['Population'].replace({'\,':''}, regex = True).astype('int')

crime_df_cleaner['Avg. Income/H/hold'] = crime_df_cleaner['Avg. Income/H/hold'].replace({'\,':''}, regex = True).astype('float')
crime_df_cleaner


,incident_report_number,crime_type,family_violence,occ_date,time_occ,zip_code,Population,Avg. Income/H/hold,National Rank,latitude,longitude,Low_access_tracts,total_tracts,Percentage_dep,food_deprived_50,food_deprived_25,food_deprived_10
0,20213651544,PUBLIC INTOXICATION,N,2021-12-31,23:50:00,78701,3855,35757.0,16509,30.267300,-97.738857,7,32,21.88,No,No,Yes
1,20213651545,ASSAULT ON PUBLIC SERVANT,N,2021-12-31,23:50:00,78701,3855,35757.0,16509,30.263739,-97.743651,7,32,21.88,No,No,Yes
2,20213651508,PUBLIC INTOXICATION,N,2021-12-31,23:21:00,78701,3855,35757.0,16509,30.269900,-97.748559,7,32,21.88,No,No,Yes
3,20225000001,BURGLARY OF VEHICLE,N,2021-12-31,21:45:00,78701,3855,35757.0,16509,30.268218,-97.738693,7,32,21.88,No,No,Yes
4,20213650837,RESISTING ARREST OR SEARCH,N,2021-12-31,15:20:00,78701,3855,35757.0,16509,30.264693,-97.737904,7,32,21.88,No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186034,20201211001,ASSAULT ON PUBLIC SERVANT,N,2020-04-30,16:54:00,78733,8716,102239.0,232,30.339078,-97.884978,0,16,0.00,No,No,No
186035,20201011302,BOATING WHILE INTOXICATED,N,2020-04-10,20:50:00,78733,8716,102239.0,232,30.353315,-97.858720,0,16,0.00,No,No,No
186036,20205014278,BURGLARY OF VEHICLE,N,2020-04-10,13:40:00,78733,8716,102239.0,232,30.339078,-97.884978,0,16,0.00,No,No,No
186037,20205013410,CHILD CUSTODY INTERFERE,N,2020-04-03,18:00:00,78733,8716,102239.0,232,30.334920,-97.870245,0,16,0.00,No,No,No


In [106]:
crime_df_cleaner.dtypes

incident_report_number            object
crime_type                        object
family_violence                   object
occ_date                  datetime64[ns]
time_occ                  datetime64[ns]
zip_code                           int64
Population                         int32
Avg. Income/H/hold               float64
National Rank                      int32
latitude                         float64
longitude                        float64
Low_access_tracts                  int64
total_tracts                       int64
Percentage_dep                   float64
food_deprived_50                  object
food_deprived_25                  object
food_deprived_10                  object
dtype: object

# Finding frequency of crime in hour increments

In [107]:
crime_times_df = crime_df_cleaner.groupby('time_occ').count()
crime_times_hour_df = crime_times_df.resample('60T').sum()
crime_times_hour_df

Time_occurance = []

for i in range(24):
          if i < 23:
                    val = f"{i}:00 - {i+1}:00"
                    Time_occurance.append(val)
          else:
                    val = f"{i}:00 - 0:00"
                    Time_occurance.append(val)

crime_times_hour_df['time_frame'] = Time_occurance
crime_times_hour_df = crime_times_hour_df.reset_index()
crime_times_hour_df = crime_times_hour_df[['time_frame', 'crime_type']]
crime_times_hour_df.rename(columns = {'time_frame': 'time_frame_by_hour', 'crime_type':'total_crimes_occured'})

,time_frame_by_hour,total_crimes_occured
0,0:00 - 1:00,12083
1,1:00 - 2:00,6788
2,2:00 - 3:00,6582
3,3:00 - 4:00,4840
4,4:00 - 5:00,3688
5,5:00 - 6:00,2991
6,6:00 - 7:00,3142
7,7:00 - 8:00,3911
8,8:00 - 9:00,5890
9,9:00 - 10:00,5778


In [108]:
crime_times_hour_df.to_csv('crime_timeframes.csv')

In [109]:
# Statistical analysis of the crime_df_cleaner
Crime_stats= crime_df_cleaner.describe()
Crime_stats.drop(columns=['zip_code','latitude', 'longitude'], axis=1, inplace=True)
Crime_stats


,Population,Avg. Income/H/hold,National Rank,Low_access_tracts,total_tracts,Percentage_dep
count,186039.000000,186039.000000,186039.000000,186039.000000,186039.000000,186039.000000
mean,29431.011315,42458.117470,14089.194862,10.696150,47.696472,22.660436
std,14494.024945,16908.494904,9004.201003,6.722376,17.372620,16.443042
min,625.000000,14740.000000,58.000000,0.000000,4.000000,0.000000
25%,19585.000000,34242.000000,8326.000000,6.000000,32.000000,13.330000
50%,30110.000000,38206.000000,13714.000000,9.000000,52.000000,23.440000
75%,42820.000000,45090.000000,18353.000000,17.000000,56.000000,28.120000
max,53044.000000,128524.000000,31414.000000,22.000000,84.000000,95.000000


# Cleaning and merging weather data

In [111]:
weather_data_df = pd.read_csv('weather_data_cleaned.csv')
weather_data_df.head()

,dt,dt_iso,temp,wind_speed,weather_main,pressure,humidity,time_occ,occ_date
0,1577836800,2020-01-01 00:00:00 +0000 UTC,54.63,1.01,Clear,1021,35,00:00:00,01/01/2020
1,1577840400,2020-01-01 01:00:00 +0000 UTC,53.10,0.00,Clouds,1021,40,01:00:00,01/01/2020
2,1577844000,2020-01-01 02:00:00 +0000 UTC,53.22,0.00,Clouds,1021,39,02:00:00,01/01/2020
3,1577847600,2020-01-01 03:00:00 +0000 UTC,53.08,1.01,Clouds,1021,40,03:00:00,01/01/2020
4,1577851200,2020-01-01 04:00:00 +0000 UTC,52.86,4.70,Clouds,1021,41,04:00:00,01/01/2020


In [116]:
#Get rid of UTC extension

timefmt_date = "%Y-%m-%d %H:%M:%S" 
weather_data_df['dt_iso'] = weather_data_df['dt_iso'].replace({'\+0000 UTC':''}, regex = True)
weather_data_df['dt_iso']= pd.to_datetime(weather_data_df['dt_iso'], format = timefmt_date, errors='coerce')
weather_data_df.head()


,dt,dt_iso,temp,wind_speed,weather_main,pressure,humidity,time_occ,occ_date
0,1577836800,2020-01-01 00:00:00,54.63,1.01,Clear,1021,35,00:00:00,01/01/2020
1,1577840400,2020-01-01 01:00:00,53.10,0.00,Clouds,1021,40,01:00:00,01/01/2020
2,1577844000,2020-01-01 02:00:00,53.22,0.00,Clouds,1021,39,02:00:00,01/01/2020
3,1577847600,2020-01-01 03:00:00,53.08,1.01,Clouds,1021,40,03:00:00,01/01/2020
4,1577851200,2020-01-01 04:00:00,52.86,4.70,Clouds,1021,41,04:00:00,01/01/2020


In [119]:
#convert time_occ back to string for simplicity sake
crime_df_cleaner['time_occ'] = [time.time() for time in crime_df_cleaner['time_occ']]

AttributeError: 'datetime.time' object has no attribute 'time'

In [121]:
# Format date time column to match with weather data
crime_df_cleaner['date_and_time'] = crime_df_cleaner['occ_date'].astype('str') + ' ' + crime_df_cleaner['time_occ'].astype('str')
timefmt_date = "%Y-%m-%d %H:%M:%S" 
crime_df_cleaner['date_and_time']= pd.to_datetime(crime_df_cleaner['date_and_time'], format = timefmt_date, errors='coerce')
crime_df_cleaner.head()


,incident_report_number,crime_type,family_violence,occ_date,time_occ,zip_code,Population,Avg. Income/H/hold,National Rank,latitude,longitude,Low_access_tracts,total_tracts,Percentage_dep,food_deprived_50,food_deprived_25,food_deprived_10,date_and_time
0,20213651544,PUBLIC INTOXICATION,N,2021-12-31,23:50:00,78701,3855,35757.0,16509,30.267300,-97.738857,7,32,21.88,No,No,Yes,2021-12-31 23:50:00
1,20213651545,ASSAULT ON PUBLIC SERVANT,N,2021-12-31,23:50:00,78701,3855,35757.0,16509,30.263739,-97.743651,7,32,21.88,No,No,Yes,2021-12-31 23:50:00
2,20213651508,PUBLIC INTOXICATION,N,2021-12-31,23:21:00,78701,3855,35757.0,16509,30.269900,-97.748559,7,32,21.88,No,No,Yes,2021-12-31 23:21:00
3,20225000001,BURGLARY OF VEHICLE,N,2021-12-31,21:45:00,78701,3855,35757.0,16509,30.268218,-97.738693,7,32,21.88,No,No,Yes,2021-12-31 21:45:00
4,20213650837,RESISTING ARREST OR SEARCH,N,2021-12-31,15:20:00,78701,3855,35757.0,16509,30.264693,-97.737904,7,32,21.88,No,No,Yes,2021-12-31 15:20:00


In [123]:
#format the crime dataframe so it can be merged with the data

# crime_format_for_weather = crime_df_cleaner.groupby(['crime_type', 'date_and_time']).count()
# crime_format_for_weather.index.get_level_values

crime_df_weather = crime_df_cleaner.groupby(['crime_type', 'date_and_time', 'incident_report_number']).count()
crime_df_weather

# Use grouper to generalize times for weather data by parcing multiple indexes 
level_values = crime_df_weather.index.get_level_values
result = crime_df_weather.groupby([level_values(i) for i in [0,2]]
                       +[pd.Grouper(freq='H', level=1)]).sum()

result
# result = result.droplevel(level=1)
result.reset_index(inplace=True)
result

crime_by_app_dt = result[['incident_report_number','crime_type', 'date_and_time']]
crime_by_app_dt.head()

,incident_report_number,crime_type,date_and_time
0,2020100120,ABUSE OF 911,2020-01-10 02:00:00
1,20201030445,ABUSE OF 911,2020-04-12 10:00:00
2,20201501388,ABUSE OF 911,2020-05-29 19:00:00
3,20201630133,ABUSE OF 911,2020-06-11 02:00:00
4,20201830315,ABUSE OF 911,2020-07-01 07:00:00


In [124]:
weather_crime_data = pd.merge(crime_by_app_dt, weather_data_df, how ='inner',left_on='date_and_time', right_on='dt_iso').drop(columns=['dt_iso','dt', 'time_occ','occ_date'])
weather_crime_data.drop_duplicates(subset=['incident_report_number'], inplace=True)
weather_crime_data


,incident_report_number,crime_type,date_and_time,temp,wind_speed,weather_main,pressure,humidity
0,2020100120,ABUSE OF 911,2020-01-10 02:00:00,71.37,3.00,Clouds,1014,76
1,2020100492,AUTO THEFT,2020-01-10 02:00:00,71.37,3.00,Clouds,1014,76
2,20205001321,BURGLARY OF RESIDENCE,2020-01-10 02:00:00,71.37,3.00,Clouds,1014,76
3,2020100130,BURGLARY OF VEHICLE,2020-01-10 02:00:00,71.37,3.00,Clouds,1014,76
4,20205001370,BURGLARY OF VEHICLE,2020-01-10 02:00:00,71.37,3.00,Clouds,1014,76
...,...,...,...,...,...,...,...,...
195473,20215006133,VIOL CITY ORDINANCE - WRECKER,2021-02-11 07:00:00,35.58,5.99,Drizzle,1018,87
195476,2021470200,VIOL OF EMERG PROTECTIVE ORDER,2021-02-16 04:00:00,15.33,3.44,Clear,1026,72
195477,20212800222,VIOL OF PROTECTIVE ORDER,2021-10-07 05:00:00,72.01,3.00,Clear,1016,58
195478,20213070172,VIOL OF PROTECTIVE ORDER,2021-11-03 05:00:00,65.71,1.01,Rain,1020,85


In [125]:
weather_crime_data.to_csv('weather_crime_data.csv')

In [126]:
crime_data_cleaned = pd.merge(crime_df_cleaner, weather_crime_data, how ='inner', on='incident_report_number')
crime_data_cleaned.drop_duplicates(subset=['incident_report_number'])

,incident_report_number,crime_type_x,family_violence,occ_date,time_occ,zip_code,Population,Avg. Income/H/hold,National Rank,latitude,...,food_deprived_25,food_deprived_10,date_and_time_x,crime_type_y,date_and_time_y,temp,wind_speed,weather_main,pressure,humidity
0,20213651544,PUBLIC INTOXICATION,N,2021-12-31,23:50:00,78701,3855,35757.0,16509,30.267300,...,No,Yes,2021-12-31 23:50:00,PUBLIC INTOXICATION,2021-12-31 23:00:00,80.80,3.0,Clouds,1003,66
1,20213651545,ASSAULT ON PUBLIC SERVANT,N,2021-12-31,23:50:00,78701,3855,35757.0,16509,30.263739,...,No,Yes,2021-12-31 23:50:00,ASSAULT ON PUBLIC SERVANT,2021-12-31 23:00:00,80.80,3.0,Clouds,1003,66
2,20213651508,PUBLIC INTOXICATION,N,2021-12-31,23:21:00,78701,3855,35757.0,16509,30.269900,...,No,Yes,2021-12-31 23:21:00,PUBLIC INTOXICATION,2021-12-31 23:00:00,80.80,3.0,Clouds,1003,66
3,20225000001,BURGLARY OF VEHICLE,N,2021-12-31,21:45:00,78701,3855,35757.0,16509,30.268218,...,No,Yes,2021-12-31 21:45:00,BURGLARY OF VEHICLE,2021-12-31 21:00:00,79.86,3.0,Clouds,1003,72
4,20213650837,RESISTING ARREST OR SEARCH,N,2021-12-31,15:20:00,78701,3855,35757.0,16509,30.264693,...,No,Yes,2021-12-31 15:20:00,RESISTING ARREST OR SEARCH,2021-12-31 15:00:00,66.92,3.0,Mist,1008,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186034,20201211001,ASSAULT ON PUBLIC SERVANT,N,2020-04-30,16:54:00,78733,8716,102239.0,232,30.339078,...,No,No,2020-04-30 16:54:00,ASSAULT ON PUBLIC SERVANT,2020-04-30 16:00:00,76.55,0.0,Clear,1021,35
186035,20201011302,BOATING WHILE INTOXICATED,N,2020-04-10,20:50:00,78733,8716,102239.0,232,30.353315,...,No,No,2020-04-10 20:50:00,BOATING WHILE INTOXICATED,2020-04-10 20:00:00,72.43,3.0,Clear,1016,62
186036,20205014278,BURGLARY OF VEHICLE,N,2020-04-10,13:40:00,78733,8716,102239.0,232,30.339078,...,No,No,2020-04-10 13:40:00,BURGLARY OF VEHICLE,2020-04-10 13:00:00,63.19,4.0,Clouds,1017,89
186037,20205013410,CHILD CUSTODY INTERFERE,N,2020-04-03,18:00:00,78733,8716,102239.0,232,30.334920,...,No,No,2020-04-03 18:00:00,CHILD CUSTODY INTERFERE,2020-04-03 18:00:00,71.28,3.0,Mist,1013,94


In [127]:
# Rearrange cleaned crime df
crime_data_cleaned.drop(columns=['crime_type_y','date_and_time_x', 'date_and_time_y'])

column_order = ['incident_report_number',
                    'crime_type_x',
                    'family_violence',
                    'occ_date',
                    'time_occ',
                    'zip_code',
                    'Population',
                    'Avg. Income/H/hold',
                    'National Rank',
                    'Percentage_dep',
                    'food_deprived_50',
                    'food_deprived_25',
                    'food_deprived_10',
                    'food_deprived_10',
                    'temp',
                    'wind_speed',
                    'weather_main',	
                    'pressure',
                    'humidity',
                    'latitude',
                    'longitude',]
crime_data_cleaned = crime_data_cleaned[column_order]

In [128]:
crime_data_cleaned

,incident_report_number,crime_type_x,family_violence,occ_date,time_occ,zip_code,Population,Avg. Income/H/hold,National Rank,Percentage_dep,...,food_deprived_25,food_deprived_10,food_deprived_10,temp,wind_speed,weather_main,pressure,humidity,latitude,longitude
0,20213651544,PUBLIC INTOXICATION,N,2021-12-31,23:50:00,78701,3855,35757.0,16509,21.88,...,No,Yes,Yes,80.80,3.0,Clouds,1003,66,30.267300,-97.738857
1,20213651545,ASSAULT ON PUBLIC SERVANT,N,2021-12-31,23:50:00,78701,3855,35757.0,16509,21.88,...,No,Yes,Yes,80.80,3.0,Clouds,1003,66,30.263739,-97.743651
2,20213651508,PUBLIC INTOXICATION,N,2021-12-31,23:21:00,78701,3855,35757.0,16509,21.88,...,No,Yes,Yes,80.80,3.0,Clouds,1003,66,30.269900,-97.748559
3,20225000001,BURGLARY OF VEHICLE,N,2021-12-31,21:45:00,78701,3855,35757.0,16509,21.88,...,No,Yes,Yes,79.86,3.0,Clouds,1003,72,30.268218,-97.738693
4,20213650837,RESISTING ARREST OR SEARCH,N,2021-12-31,15:20:00,78701,3855,35757.0,16509,21.88,...,No,Yes,Yes,66.92,3.0,Mist,1008,91,30.264693,-97.737904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186034,20201211001,ASSAULT ON PUBLIC SERVANT,N,2020-04-30,16:54:00,78733,8716,102239.0,232,0.00,...,No,No,No,76.55,0.0,Clear,1021,35,30.339078,-97.884978
186035,20201011302,BOATING WHILE INTOXICATED,N,2020-04-10,20:50:00,78733,8716,102239.0,232,0.00,...,No,No,No,72.43,3.0,Clear,1016,62,30.353315,-97.858720
186036,20205014278,BURGLARY OF VEHICLE,N,2020-04-10,13:40:00,78733,8716,102239.0,232,0.00,...,No,No,No,63.19,4.0,Clouds,1017,89,30.339078,-97.884978
186037,20205013410,CHILD CUSTODY INTERFERE,N,2020-04-03,18:00:00,78733,8716,102239.0,232,0.00,...,No,No,No,71.28,3.0,Mist,1013,94,30.334920,-97.870245


In [ ]:
# crime_format_for_weather_counts = crime_df_weather.resample('60T').sum()
# crime_format_for_weather_counts =  crime_format_for_weather_counts.reset_index()
# crime_format_for_weather_counts = crime_format_for_weather_counts[['date_and_time', 'crime_type']]
# crime_format_for_weather_counts = crime_format_for_weather_counts.rename(columns={'crime_type':'total_crimes'})
# crime_format_for_weather_counts

In [ ]:
# weather_crime_df = pd.merge(crime_format_for_weather, weather_data_df, how="inner", left_on=['date_and_time'], right_on=['dt_iso']).drop(columns=['dt', 'dt_iso'])
# weather_crime_df

In [129]:
# crime_df_cleaner.to_csv('crime_data_cleaned_expanded.csv', index=False)
# crime_by_zipcode_df.to_csv("crime_zipcode_data_expanded.csv")
# crime_times_hour_df.to_csv("crimes_by_hour.csv")
crime_data_cleaned.to_csv('crime_data_cleaned.csv')